In [2]:
import os
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts
import whisper
import requests
from pyunsplash import PyUnsplash
load_dotenv()
import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [34]:
def download_images(text: str, counts: int = 1):
    for i in range(counts):
        pu = PyUnsplash(api_key=os.getenv('UNSPLASH_ACCESS_KEY'))
        photos = pu.photos(type_ = 'random', count = 1, query = text)
        [photo] = photos.entries
        print(photo.id, photo.link_download)
        response = requests.get(photo.link_download, allow_redirects=True)

        if os.path.exists('images'):
            open(f'./images/{text}{i}.jpg', 'wb').write(response.content)
        else:
            os.mkdir('images')
            open(f'images/{text}{i}.jpg', 'wb').write(response.content)

In [35]:
download_images('Dog', counts = 5)

KFJuCzJiQYU https://unsplash.com/photos/KFJuCzJiQYU/download?ixid=M3w1NzQ1NjJ8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MTA0NDA5ODd8
Ytp_K5bwa0Q https://unsplash.com/photos/Ytp_K5bwa0Q/download?ixid=M3w1NzQ1NjJ8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MTA0NDA5OTB8
pOUA8Xay514 https://unsplash.com/photos/pOUA8Xay514/download?ixid=M3w1NzQ1NjJ8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MTA0NDA5OTF8
EpRAM95thHU https://unsplash.com/photos/EpRAM95thHU/download?ixid=M3w1NzQ1NjJ8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MTA0NDA5OTR8
tUCvnMOvXFQ https://unsplash.com/photos/tUCvnMOvXFQ/download?ixid=M3w1NzQ1NjJ8MHwxfHJhbmRvbXx8fHx8fHx8fDE3MTA0NDA5OTV8


In [11]:
for i in range(5):
    with open(f'./stories/story_{i}.txt', 'r') as f:
        text = f.read()

def eleven_speak(text, file_name):
    engine = tts.init()
    engine.setProperty('rate', 200)
    engine.save_to_file(text, f'audio/{file_name}.mp3')
    engine.runAndWait()

In [13]:
for j in range(5):
    eleven_speak(text, f'story_{j}')

In [14]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    data = model.transcribe(audio_file, word_timestamps=True)

    start = [data['segments'][i]['words'][j]['start'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    end = [data['segments'][i]['words'][j]['end'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    text = [data['segments'][i]['words'][j]['word'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]

    return [start, end, text]

In [ ]:
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.editor import CompositeAudioClip, concatenate_audioclips

def generate_video():
    image_list = ['Cat', 'Dog', 'Car', 'House', 'Tree']
    for i in range(stories):
        with open(f'stories/story_{i}.txt', 'r') as file:
            print(f'Generating video for story {i}')
            text = file.read()
        
        words = text.split(' ')

        generator = lambda txt: TextClip(txt, 
                                    fontsize=70, 
                                    color='white', 
                                    bg_color='none', 
                                    font='Arial-Bold',
                                    method = 'caption',
                                    size = (1920, 1080))

        image_clips = []

        # Fetching images from Unsplash
        for j in range(len(words) // 20 + 1):
            images = get_images(image_list[j])
            image_path = f'bg_{i}_{j}.jpg'
            with open(image_path, 'wb') as file:


        speech = speech_to_text(f'audio/story_{i}.mp3')
        print(f'Accessing speech for story {i}')
        subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]

        subtitles = SubtitlesClip(subs, generator)
        audio_clip = AudioFileClip(f'audio/story_{i}.mp3')
        video = VideoFileClip('../assets/BG.mp4')
        final_complete = CompositeVideoClip([video.set_duration(audio_clip.duration), subtitles.set_position(('center', 'center'))])
        final_complete = final_complete.set_audio(audio_clip)
        final_complete.write_videofile(f"output_{i}.mp4" , fps=24, bitrate = '1000', preset = 'fast', codec = 'h264_nvenc')
        final_complete.close()